In [1]:
# Welcome to Langchain Deep Dive

In [2]:
#Example #1 — “Chat history only” (minimal, no memory wrapper)

In [3]:
# Minimal dependencies (Colab-friendly)
!pip install -q langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.7 MB/s eta 0:00:00


In [11]:


from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from google.colab import userdata
import os

# Set your API key in Colab once with:
# from google.colab import userdata; userdata.set("OPENAI_API_KEY", "sk-...")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

#llm = ChatOpenAI()  # uses OpenAI chat model via langchain-openai

# Set your model + temperature here
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)


llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.0)

# Start with a system instruction and an empty conversation
history = [SystemMessage(content="You are a helpful, concise assistant.")]

print("💬 Chat started! Type 'exit' to quit.\n")
while True:
    user_text = input("You: ")
    if user_text.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Add the user's message to history
    history.append(HumanMessage(content=user_text))

    # Ask the model, passing the full message list (history)
    response = llm.invoke(history)

    # Add the assistant's reply to history and print it
    history.append(AIMessage(content=response.content))
    print("Bot:", response.content)


💬 Chat started! Type 'exit' to quit.

You: hello this is Sandip 
Bot: Hello Sandip! How can I assist you today?
You: who is Big B
Bot: "Big B" is a popular nickname for Amitabh Bachchan, a legendary Indian actor and film producer known for his work in Bollywood.
You: what did I ask
Bot: You asked, "who is Big B."
You: exit
👋 Goodbye!


In [7]:
# Example #2  — Memory via RunnableWithMessageHistory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1) Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.25)

# 2) Prompt (history + current input)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),     # <- message objects go here
    ("human", "{input}")                # <- the latest user turn
])

# 3) Chain
chain = prompt | llm

# 4) History store (implements BaseChatMessageHistory)
class SimpleChatHistory(BaseChatMessageHistory):
    def __init__(self): self._messages = []
    @property
    def messages(self): return self._messages
    def add_user_message(self, text): self._messages.append(HumanMessage(content=text))
    def add_ai_message(self, text): self._messages.append(AIMessage(content=text))
    def add_message(self, m): self._messages.append(m)
    def add_messages(self, ms): self._messages.extend(ms)
    def clear(self): self._messages.clear()

# Wrap chain with memory
chat = RunnableWithMessageHistory(
    chain,
    lambda session_id: SimpleChatHistory(),  # return a history object per session
    input_messages_key="input",
    history_messages_key="history"
)

# Use it
resp = chat.invoke({"input": "Hi!"}, config={"configurable": {"session_id": "s1"}})
print(resp.content)


In [ ]:
# Example # 3

In [10]:
# Install (Colab)
!pip install -q langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from google.colab import userdata
import os
import re

# ── Config ─────────────────────────────────────────────────────────────────────
# Set API key in Colab once:  userdata.set("OPENAI_API_KEY", "sk-...")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

MODEL = "gpt-4.1-nano"     # pick a ChatGPT-compatible model
TEMPERATURE = 0.25        # 0.0–0.7 is a good working range

# ── LLM ─────────────────────────────────────────────────────────────────────────
llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE)

# ── Prompt: include full message history AND a clean list of prior user questions
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant.\n"
     "Below is the conversation so far (messages):"),
    MessagesPlaceholder(variable_name="history"),
    ("system",
     "The user's previous questions (verbatim) are:\n{user_questions}\n"
     "If the user asks what they asked earlier, list those questions as bullet points, "
     "without adding extra ones."),
    ("human", "{input}")
])

chain = prompt | llm

# ── Minimal in-memory chat history ──────────────────────────────────────────────
class SimpleChatHistory(BaseChatMessageHistory):
    def __init__(self):
        self._messages: list[BaseMessage] = []

    @property
    def messages(self):
        return self._messages

    def add_user_message(self, content: str):
        self._messages.append(HumanMessage(content=content))

    def add_ai_message(self, content: str):
        self._messages.append(AIMessage(content=content))

    def add_message(self, message: BaseMessage):
        self._messages.append(message)

    def add_messages(self, messages: list[BaseMessage]):
        self._messages.extend(messages)

    def clear(self):
        self._messages.clear()

# Keep one history object per session id
_histories = {}
def get_history(session_id: str) -> SimpleChatHistory:
    if session_id not in _histories:
        _histories[session_id] = SimpleChatHistory()
    return _histories[session_id]

# Wrap chain with message history
chat_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: get_history(session_id),
    input_messages_key="input",
    history_messages_key="history",
)

# Helper: build a clean bullet list of prior user questions from history
QUESTION_LIKE = re.compile(r".*\?$|^(who|what|when|where|why|how|do|does|did|can|could|should|is|are|was|were)\b", re.I)

def prior_user_questions(history: SimpleChatHistory) -> str:
    qs = []
    for m in history.messages:
        if isinstance(m, HumanMessage):
            text = m.content.strip()
            # treat anything that looks like a question as a question
            if text.endswith("?") or QUESTION_LIKE.match(text):
                qs.append(text)
    if not qs:
        return "- (no prior user questions yet)"
    return "\n".join(f"- {q}" for q in qs)

# ── Chat loop ───────────────────────────────────────────────────────────────────
print("💬 Chat started! Type 'exit' to quit.\n")
SESSION_ID = "demo-session"

while True:
    user_text = input("You: ")
    if user_text.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Build the user-questions summary from current history
    hist = get_history(SESSION_ID)
    user_qs = prior_user_questions(hist)

    # Invoke with both the normal input and the computed {user_questions}
    response = chat_with_memory.invoke(
        {"input": user_text, "user_questions": user_qs},
        config={"configurable": {"session_id": SESSION_ID}},
    )
    print("Bot:", response.content)


💬 Chat started! Type 'exit' to quit.

You: hi I am Sandip
Bot: Hello Sandip! How can I assist you today?
You: who is SRK
Bot: SRK stands for Shah Rukh Khan, a famous Indian actor, film producer, and television personality. He is widely regarded as one of the biggest stars in Bollywood and has appeared in numerous popular movies. Would you like to know more about him?
You: Who is Big B
Bot: Big B is a popular nickname for Amitabh Bachchan, a legendary Indian actor, film producer, and television host. He is considered one of the greatest and most influential actors in the history of Indian cinema. Would you like to know more about him?
You: Who is God of Cricket in India
Bot: The title "God of Cricket" in India is commonly attributed to Sachin Tendulkar. He is regarded as one of the greatest cricketers of all time and has set numerous records in international cricket. Would you like more information about him?
You: what all I ask u
Bot: Here are the questions you asked earlier:
- who is 

In [13]:
# Embeddings , Vectors and FAISS

In [14]:
# Example 1 - Simple Embeddings

In [15]:
# Install
!pip install -q langchain-openai

# Imports
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
import os

# 1) Put your key once in Colab (in a separate cell):
# from google.colab import userdata
# userdata.set("OPENAI_API_KEY", "sk-...")

# 2) Load key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 3) Create the embeddings model
#   - "text-embedding-3-small": fast, cheap, 1536-dim
#   - "text-embedding-3-large": higher quality, 3072-dim
emb = OpenAIEmbeddings(model="text-embedding-3-small")

# 4) Take input and embed
text = input("Enter text to embed: ").strip()
vec = emb.embed_query(text)  # list[float]

# 5) Show results
print(f"\nText: {text!r}")
print(f"Vector length (dimension): {len(vec)}")
print("First 8 numbers:", [round(x, 6) for x in vec[:8]])


Enter text to embed: hello I am Sandip, live in Pune

Text: 'hello I am Sandip, live in Pune'
Vector length (dimension): 1536
First 8 numbers: [0.016876, -0.005168, 0.026773, 0.020525, -0.027236, -0.033387, -0.012087, 0.0159]


In [17]:
# Example 2 - Create + STORE embeddings (no search)

In [19]:
# ============================================
# Basic: Create + STORE embeddings (no search)
# ============================================

# 0) Install deps
!pip install -q langchain-openai faiss-cpu numpy pandas

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
import os, json, numpy as np, pandas as pd
import faiss  # direct FAISS usage

# Load key
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "❌ OPENAI_API_KEY not found in Colab userdata.\n"
        "Set it once with:\n"
        "from google.colab import userdata\n"
        "userdata.set('OPENAI_API_KEY', 'sk-...')"
    )
os.environ["OPENAI_API_KEY"] = api_key

# 2) Collect texts (user input or defaults)
raw = input(
    "Enter texts separated by '||' (or press Enter to use defaults):\n"
).strip()

if raw:
    texts = [t.strip() for t in raw.split("||") if t.strip()]
else:
    texts = [
        "MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.",
        "SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.",
        "Amitabh Bachchan is nicknamed Big B in Indian cinema.",
        "Shah Rukh Khan is popularly called King Khan.",
    ]

print(f"\n📝 {len(texts)} texts to embed.\n")

# 3) Create embeddings (vectors)
emb_model = "text-embedding-3-small"  # 1536-dim; switch to -large for 3072-dim
emb = OpenAIEmbeddings(model=emb_model)

# embed_documents -> list of vectors (one per text)
vectors = emb.embed_documents(texts)  # list[list[float]]
dim = len(vectors[0]) if vectors else 0

print(f"✅ Created {len(vectors)} embeddings with dimension = {dim} (model: {emb_model}).\n")

# 4) SHOW what we stored (table preview)
rows = []
for i, (t, v) in enumerate(zip(texts, vectors), start=1):
    rows.append({
        "id": f"doc-{i}",
        "text": t,
        "dim": len(v),
        "preview(first_8_dims)": [round(x, 6) for x in v[:8]]
    })
df = pd.DataFrame(rows)
print(df.to_string(index=False))

# 5) STORE embeddings plainly (files you can inspect)
#    - vectors.npy: the raw numeric vectors (float32)
#    - texts.jsonl: metadata (id + original text)
vec_arr = np.array(vectors, dtype="float32")
np.save("vectors.npy", vec_arr)
with open("texts.jsonl", "w", encoding="utf-8") as f:
    for i, t in enumerate(texts, start=1):
        f.write(json.dumps({"id": f"doc-{i}", "text": t}, ensure_ascii=False) + "\n")

print("\n💾 Saved raw vectors to ./vectors.npy and text metadata to ./texts.jsonl")

# 6) STORE embeddings in a FAISS index (no search yet)
#    Build an L2 index and add our vectors.
if len(vec_arr) > 0:
    index = faiss.IndexFlatL2(dim)  # exact L2 index
    index.add(vec_arr)              # add all vectors
    print(f"📦 FAISS index created. ntotal = {index.ntotal} vectors.")

    # (Optional) Save FAISS index to disk
    faiss.write_index(index, "index.faiss")
    print("💾 Saved FAISS index to ./index.faiss")

    # (Optional) Show how to reload it later
    reloaded = faiss.read_index("index.faiss")
    print(f"🔁 Reloaded FAISS index. ntotal = {reloaded.ntotal} vectors.")
else:
    print("No vectors created; FAISS index not built.")


Enter texts separated by '||' (or press Enter to use defaults):


📝 4 texts to embed.

✅ Created 4 embeddings with dimension = 1536 (model: text-embedding-3-small).

   id                                                             text  dim                                                               preview(first_8_dims)
doc-1 MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain. 1536  [0.108236, -0.016901, 0.041789, 0.037184, -0.008478, 0.011751, 0.037075, 0.007594]
doc-2     SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'. 1536  [0.047056, 0.011706, 0.035647, 0.005719, -0.037854, -0.018892, 0.030116, 0.020833]
doc-3            Amitabh Bachchan is nicknamed Big B in Indian cinema. 1536  [0.052879, -0.021684, 0.005693, 0.058494, -0.06329, -0.003692, 0.026152, 0.047099]
doc-4                    Shah Rukh Khan is popularly called King Khan. 1536 [0.016203, -0.09365, -0.018317, 0.037572, 0.015276, -0.009722, -0.006646, 0.055001]

💾 Saved raw vectors to ./vectors.

In [21]:
# Example 3 - Tiny Search

In [25]:
# ============================================
# Tiny similarity search with FAISS (Colab)
# ============================================

# 0) Install deps (if not already)
!pip install -q langchain-openai faiss-cpu numpy

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
import os, json, numpy as np, faiss, sys

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    print("❌ OPENAI_API_KEY not found. Set it with:\n"
          "from google.colab import userdata\n"
          "userdata.set('OPENAI_API_KEY', 'sk-...')\n")
    sys.exit(1)
os.environ["OPENAI_API_KEY"] = api_key

# 2) Try loading previously saved artifacts (from the “store” step)
INDEX_PATH = "index.faiss"
TEXTS_PATH = "texts.jsonl"
EMB_MODEL = "text-embedding-3-small"    # must match the model used to build the index

def load_texts_jsonl(path):
    items, ids = [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            ids.append(obj.get("id"))
            items.append(obj.get("text"))
    return ids, items

need_rebuild = False
if os.path.exists(INDEX_PATH) and os.path.exists(TEXTS_PATH):
    try:
        index = faiss.read_index(INDEX_PATH)
        doc_ids, texts = load_texts_jsonl(TEXTS_PATH)
        print(f"✅ Loaded FAISS index (ntotal={index.ntotal}) and {len(texts)} texts.")
    except Exception as e:
        print("⚠️ Could not load previous index/metadata. Will rebuild a small demo index.\nReason:", e)
        need_rebuild = True
else:
    need_rebuild = True

# 3) If needed, rebuild a small index (self-contained)
if need_rebuild:
    print("🔧 Rebuilding a small demo index locally...")
    texts = [
        "MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.",
        "SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.",
        "Amitabh Bachchan is nicknamed Big B in Indian cinema.",
        "Shah Rukh Khan is popularly called King Khan.",
        "Virat Kohli is an Indian cricketer known for batting consistency.",
        "Rohit Sharma is known for explosive batting in limited-overs."
    ]
    emb = OpenAIEmbeddings(model=EMB_MODEL)
    vecs = emb.embed_documents(texts)           # list[list[float]]
    vec_arr = np.array(vecs, dtype="float32")   # (N, dim)
    dim = vec_arr.shape[1]

    index = faiss.IndexFlatL2(dim)              # exact L2 index
    index.add(vec_arr)

    # save artifacts for later use
    faiss.write_index(index, INDEX_PATH)
    with open(TEXTS_PATH, "w", encoding="utf-8") as f:
        for i, t in enumerate(texts, start=1):
            f.write(json.dumps({"id": f"doc-{i}", "text": t}, ensure_ascii=False) + "\n")

    print(f"✅ Built and saved demo index. ntotal={index.ntotal}, dim={dim}")

# 4) Run a tiny search
emb = OpenAIEmbeddings(model=EMB_MODEL)  # same model as used for the index
query = input("\nEnter your search query (e.g., 'Who is King Khan?'): ").strip()
if not query:
    query = "Who is King Khan?"
    print(f"(Using default) {query}")

q_vec = np.array([emb.embed_query(query)], dtype="float32")  # shape (1, dim)

k = 3  # top-k results
distances, indices = index.search(q_vec, k)   # shapes: (1, k), (1, k)
dist_list = distances[0].tolist()
idx_list = indices[0].tolist()

# load texts (if not already loaded)
if 'texts' not in globals():
    _, texts = load_texts_jsonl(TEXTS_PATH)

print(f"\n🔎 Top-{k} results for: {query!r}  (Lower distance = closer match)\n")
for rank, (idx, dist) in enumerate(zip(idx_list, dist_list), start=1):
    if idx == -1:
        continue  # FAISS returns -1 if not enough results
    # idx corresponds to the position of the text used when building the index
    print(f"{rank}. [distance={dist:.4f}]  {texts[idx]}")

print("\n✅ Done.")


✅ Loaded FAISS index (ntotal=4) and 4 texts.

Enter your search query (e.g., 'Who is King Khan?'): Who is MSD

🔎 Top-3 results for: 'Who is MSD'  (Lower distance = closer match)

1. [distance=0.8261]  MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.
2. [distance=1.5741]  SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
3. [distance=1.8090]  Amitabh Bachchan is nicknamed Big B in Indian cinema.

✅ Done.


In [26]:
# Example 4 - Cosine Similarity

In [27]:
# ============================================================
# Cosine similarity search with FAISS (normalized vectors)
# ============================================================

# 0) Install
!pip install -q langchain-openai faiss-cpu numpy

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
import os, json, numpy as np, faiss, sys

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    print("❌ OPENAI_API_KEY not found. Set it with:\n"
          "from google.colab import userdata\n"
          "userdata.set('OPENAI_API_KEY', 'sk-...')\n")
    sys.exit(1)
os.environ["OPENAI_API_KEY"] = api_key

EMB_MODEL = "text-embedding-3-small"  # same model must be used for index & queries

# 2) Data to index (you can replace with your own)
texts = [
    "MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.",
    "SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.",
    "Amitabh Bachchan is nicknamed Big B in Indian cinema.",
    "Shah Rukh Khan is popularly called King Khan.",
    "Virat Kohli is an Indian cricketer known for batting consistency.",
    "Rohit Sharma is known for explosive batting in limited-overs."
]

# 3) Embed and build a cosine index (normalize + IP)
emb = OpenAIEmbeddings(model=EMB_MODEL)

def l2_normalize_rows(mat: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(mat, axis=1, keepdims=True) + 1e-12
    return mat / norms

# Embed documents
doc_vecs = emb.embed_documents(texts)            # list[list[float]]
doc_arr = np.array(doc_vecs, dtype="float32")    # (N, dim)
doc_arr = l2_normalize_rows(doc_arr)             # normalize for cosine

dim = doc_arr.shape[1]
index = faiss.IndexFlatIP(dim)                   # inner product ≈ cosine (after normalization)
index.add(doc_arr)                               # add normalized vectors
print(f"✅ Cosine index ready. ntotal={index.ntotal}, dim={dim}")

# Save small metadata file so we can map results back to text
with open("texts.jsonl", "w", encoding="utf-8") as f:
    for i, t in enumerate(texts):
        f.write(json.dumps({"id": i, "text": t}, ensure_ascii=False) + "\n")

# 4) Query loop
def load_texts(path="texts.jsonl"):
    items = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            items.append(json.loads(line)["text"])
    return items

stored_texts = load_texts()

print("\n💬 Type your query (e.g., 'Who is King Khan?'). Type 'exit' to quit.")
while True:
    q = input("Query: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    if not q:
        q = "Who is King Khan?"
        print(f"(Using default) {q}")

    # Embed and normalize the query
    q_vec = np.array([emb.embed_query(q)], dtype="float32")
    q_vec = l2_normalize_rows(q_vec)

    k = 3
    sims, idxs = index.search(q_vec, k)   # sims in [0..1] after normalization
    sims, idxs = sims[0], idxs[0]

    print(f"\n🔎 Top-{k} cosine matches for: {q!r}")
    for rank, (i, s) in enumerate(zip(idxs, sims), 1):
        if i == -1: continue
        print(f"{rank}. score={s:.4f}  |  {stored_texts[i]}")
    print("")


✅ Cosine index ready. ntotal=6, dim=1536

💬 Type your query (e.g., 'Who is King Khan?'). Type 'exit' to quit.
Query: who is Virat Kohli

🔎 Top-3 cosine matches for: 'who is Virat Kohli'
1. score=0.6832  |  Virat Kohli is an Indian cricketer known for batting consistency.
2. score=0.4041  |  Rohit Sharma is known for explosive batting in limited-overs.
3. score=0.3921  |  MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.

Query: who is King Khan

🔎 Top-3 cosine matches for: 'who is King Khan'
1. score=0.6424  |  Shah Rukh Khan is popularly called King Khan.
2. score=0.3208  |  Amitabh Bachchan is nicknamed Big B in Indian cinema.
3. score=0.3035  |  Virat Kohli is an Indian cricketer known for batting consistency.

Query: who is Rohit Sharma

🔎 Top-3 cosine matches for: 'who is Rohit Sharma'
1. score=0.6800  |  Rohit Sharma is known for explosive batting in limited-overs.
2. score=0.4568  |  SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
3. score=0.3934  |  

In [31]:
"""plug in a GPT model and do a tiny, clean RAG (Retrieval‑Augmented Generation): retrieve the most relevant snippets from your FAISS index and ask the model to answer using only that context.

Below is a single Colab‑ready code block that:

builds a small FAISS index (you can paste your own text),

wraps it as a retriever,

formats retrieved context,

calls a GPT model to answer from that context,

runs an interactive Q&A loop.
"""


"""
What this gives us:

Embeddings + FAISS turn your reference text into a searchable index.

A retriever fetches the top-k relevant chunks for each question.

A prompt instructs the model to answer only from the retrieved context, avoiding hallucinations.

A tiny interactive loop to ask multiple questions.
"""


'\nWhat this gives us: \n\nEmbeddings + FAISS turn your reference text into a searchable index.\n\nA retriever fetches the top-k relevant chunks for each question.\n\nA prompt instructs the model to answer only from the retrieved context, avoiding hallucinations.\n\nA tiny interactive loop to ask multiple questions.\n'

In [33]:
# Example 5 - GPT Model Plug In

In [30]:
# ============================================================
# Tiny RAG: GPT answers from FAISS-retrieved context (Colab)
# ============================================================

# 0) Install packages
!pip install -q langchain-openai langchain-community faiss-cpu langchain-text-splitters

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata
import os

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "❌ OPENAI_API_KEY not found. Set once with:\n"
        "from google.colab import userdata\n"
        "userdata.set('OPENAI_API_KEY', 'sk-...')"
    )
os.environ["OPENAI_API_KEY"] = api_key

# 2) Paste your own source text (or press Enter to use defaults)
raw = input(
    "Paste your reference text (or press Enter to use a default cricket sample):\n"
).strip()

if not raw:
    raw = """
Mahendra Singh Dhoni (MSD) captained India and won the 2007 T20 World Cup and the 2011 ODI World Cup.
Sachin Ramesh Tendulkar (SRT) is widely called the 'God of Cricket' for his batting records.
Amitabh Bachchan, nicknamed Big B, is a legendary actor in Indian cinema.
Shah Rukh Khan, often called King Khan, is one of the most popular Bollywood actors worldwide.
Virat Kohli is renowned for batting consistency across formats.
Rohit Sharma is known for explosive batting and captaincy in limited-overs cricket.
"""

# 3) Split into chunks (simple defaults work fine to start)
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=60)
docs = splitter.create_documents([raw])

# 4) Build vector store + retriever
emb = OpenAIEmbeddings(model="text-embedding-3-small")   # fast & good
vs = FAISS.from_documents(docs, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

# 5) Define the RAG prompt
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n"
    "If the answer is not in the context, say you don't know.\n"
    "Answer:"
)

# 6) Helper to format retrieved docs
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# 7) LLM + chain
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8) Interactive Q&A
print("\n💬 Ask questions about your text. Type 'exit' to quit.")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Optional: peek at retrieved chunks (uncomment to see)
    # retrieved = retriever.get_relevant_documents(q)
    # print("\n[DEBUG] Retrieved context:\n", format_docs(retrieved), "\n")

    ans = rag_chain.invoke(q)
    print("A:", ans, "\n")


Paste your reference text (or press Enter to use a default cricket sample):


💬 Ask questions about your text. Type 'exit' to quit.
Q: who is SRT
A: SRT is Sachin Ramesh Tendulkar. 

Q: who is MSD
A: MSD is Mahendra Singh Dhoni. 

Q: who is King Khan
A: Shah Rukh Khan is often called King Khan. 

Q: who is Sourav Ganguly
A: I don't know. 

Q: who is virat Kohli
A: Virat Kohli is renowned for batting consistency across formats. 

Q: who is Rohit Sharma
A: Rohit Sharma is known for explosive batting and captaincy in limited-overs cricket. 

Q: who is Jasprit Bumrah
A: The context does not provide information about Jasprit Bumrah. 

Q: exit
👋 Goodbye!


In [34]:
# Example 6 - Chunking + Metadata + Citations

In [35]:
# --- Install ---
!pip install -q langchain-openai langchain-community langchain-text-splitters faiss-cpu

# --- Imports & API key ---
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata
from langchain.schema import Document
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- 1) Source text (replace with your own) ---
raw_text = """
Mahendra Singh Dhoni (MSD) captained India and won the 2007 T20 World Cup and the 2011 ODI World Cup.
Sachin Ramesh Tendulkar (SRT) is widely called the 'God of Cricket' for his batting records.
Amitabh Bachchan, nicknamed Big B, is a legendary actor in Indian cinema.
Shah Rukh Khan, often called King Khan, is one of the most popular Bollywood actors worldwide.
Virat Kohli is renowned for batting consistency across formats.
Rohit Sharma is known for explosive batting and captaincy in limited-overs cricket.
"""

# --- 2) Chunk with metadata (source + chunk_id) ---
splitter = RecursiveCharacterTextSplitter(chunk_size=280, chunk_overlap=80)
chunks = splitter.split_text(raw_text)
docs = [
    Document(page_content=chunk, metadata={"source": "notes.txt", "chunk_id": i})
    for i, chunk in enumerate(chunks)
]

# --- 3) Build vector store + retriever ---
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(docs, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

# --- 4) Prompt that asks model to cite chunks used ---
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "If the answer is not in the context, say you don't know.\n"
    "At the end, list citations as [source#chunk_id] for each chunk you used."
)

def format_docs(docs):
    # Include metadata so the model can cite properly
    lines = []
    for d in docs:
        src = d.metadata.get("source", "unknown")
        cid = d.metadata.get("chunk_id", "na")
        lines.append(f"[{src}#{cid}] {d.page_content}")
    return "\n\n".join(lines)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Ask questions (type 'exit' to quit):")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    print("A:", rag_chain.invoke(q), "\n")


Ask questions (type 'exit' to quit):
Q: who is Virat Kohli
A: Virat Kohli is renowned for batting consistency across formats. [source#2] 

Q: who is Rohit Sharma
A: Rohit Sharma is known for explosive batting and captaincy in limited-overs cricket. [source#2] 

Q: who is MSD
A: MSD refers to Mahendra Singh Dhoni, who captained India and won the 2007 T20 World Cup and the 2011 ODI World Cup. 

Citations: [notes.txt#0] 

Q: who is SRT
A: SRT refers to Sachin Ramesh Tendulkar, who is widely called the 'God of Cricket' for his batting records. [source#0] 

Q: who is Big B
A: Big B is Amitabh Bachchan, a legendary actor in Indian cinema. 

Citations: [notes.txt#1] 

Q: Who is Rahul Dravid
A: I don't know. 

Citations: None 

Q: exit
👋 Goodbye!


In [43]:
# Example 7 - Persistence (Save/Load FAISS)

In [42]:
# ================================
# FAISS persistence
# ================================

# 0) Install
!pip install -q langchain-openai langchain-community faiss-cpu

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import os, sys

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    print("❌ OPENAI_API_KEY not found.\n"
          "Set it once with:\n"
          "from google.colab import userdata\n"
          "userdata.set('OPENAI_API_KEY', 'sk-...')\n")
    sys.exit(1)
os.environ["OPENAI_API_KEY"] = api_key

# 2) Small corpus
texts = [
    "MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.",
    "MSD won multiple world cups for India"
    "MSD is the first captain ever to win a T20 world cup"
    "SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.",
    "Amitabh Bachchan is nicknamed Big B in Indian cinema.",
    "Shah Rukh Khan is popularly called King Khan.",
    "Virat Kohli is known for batting consistency across formats.",
    "Rohit Sharma is known for explosive batting in limited-overs."
]

# 3) Build vector store (embeddings + FAISS)
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, embedding=emb)
print("✅ Built FAISS index with", len(texts), "texts.")

# 4) Save to disk (persistence)
save_dir = "/content/my_faiss_plain"
vs.save_local(save_dir)
print("💾 Saved index to:", save_dir)

# 5) Load it back (later or in a new runtime)
vs2 = FAISS.load_local(save_dir, emb, allow_dangerous_deserialization=True)
print("🔁 Reloaded index. Size =", len(vs2.index_to_docstore_id))

# 6) Plain similarity search (no MMR)
retriever_plain = vs2.as_retriever(search_kwargs={"k": 3})  # default: top-k similarity

def search(query: str, k: int = 3):
    docs = retriever_plain.get_relevant_documents(query)
    print(f"\n🔎 Top-{k} results for: {query!r}")
    for i, d in enumerate(docs[:k], 1):
        print(f"{i}. {d.page_content}")

# 7) Try a couple of queries
search("Who is King Khan?", k=3)
search("Who is MSD?", k=3)
search("Who is called the God of Cricket?", k=3)



✅ Built FAISS index with 6 texts.
💾 Saved index to: /content/my_faiss_plain
🔁 Reloaded index. Size = 6

🔎 Top-3 results for: 'Who is King Khan?'
1. Shah Rukh Khan is popularly called King Khan.
2. Amitabh Bachchan is nicknamed Big B in Indian cinema.
3. Virat Kohli is known for batting consistency across formats.

🔎 Top-3 results for: 'Who is MSD?'
1. MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.
2. MSD won multiple world cups for IndiaMSD is the first captain ever to win a T20 world cupSRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
3. Rohit Sharma is known for explosive batting in limited-overs.

🔎 Top-3 results for: 'Who is called the God of Cricket?'
1. MSD won multiple world cups for IndiaMSD is the first captain ever to win a T20 world cupSRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
2. MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.
3. Rohit Sharma is known for explosive batting in limited-overs.


In [ ]:
# Incorrect answers as cosine similarity is not implemented.

In [39]:
# --- Install (if not already) ---
!pip install -q langchain-openai langchain-community faiss-cpu

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Suppose you already have a FAISS `vs` from the previous cell.
# If not, quickly rebuild from a tiny set:
docs = [
    "MSD won the 2011 ODI World Cup as captain.",
    "SRT is called the God of Cricket.",
    "King Khan refers to Shah Rukh Khan."
]
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(docs, embedding=emb)

# --- Save ---
save_dir = "/content/my_faiss_rag"
vs.save_local(save_dir)
print("💾 Saved to:", save_dir)

# --- Load in a new session (or just to test) ---
vs2 = FAISS.load_local(save_dir, emb, allow_dangerous_deserialization=True)
print("🔁 Reloaded. Size =", len(vs2.index_to_docstore_id))

# --- Turn on MMR for diverse top-k ---
retriever_mmr = vs2.as_retriever(
    search_kwargs={
        "k": 3,        # results to return
        "fetch_k": 20, # candidates to consider
        "lambda_mult": 0.5  # 0→diversity, 1→similarity; try 0.2–0.7
    },
    search_type="mmr"
)

# Quick test query
ctx_docs = retriever_mmr.get_relevant_documents("Tell me about World Cups")
for i, d in enumerate(ctx_docs, 1):
    print(f"{i}.", d.page_content)


💾 Saved to: /content/my_faiss_rag
🔁 Reloaded. Size = 3
1. MSD won the 2011 ODI World Cup as captain.
2. King Khan refers to Shah Rukh Khan.
3. SRT is called the God of Cricket.


In [44]:
# Example 8 - RAG Implementation - supporting PDF / URL / TXT with a simple switch.

"""
It:

loads the document(s)

chunks the text

builds a FAISS index (cosine similarity)

runs a small RAG loop with citations
"""


'\nIt:\n\nloads the document(s)\n\nchunks the text\n\nbuilds a FAISS index (cosine similarity)\n\nruns a small RAG loop with citations \n'

In [50]:
# =========================================
# RAG over PDF / URL / TXT (Colab-ready)
# =========================================

# 0) Install
!pip install -q langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf bs4

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from google.colab import userdata
import os

# API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 2) === Choose your source here ===
SOURCE_TYPE = "txt"   # "pdf" | "url" | "txt"

# Provide ONE of these depending on SOURCE_TYPE
PDF_PATH = "/content/Databricks-Big-Book-Of-GenAI-FINAL.pdf"                       # e.g., upload via Colab sidebar, then set path
URLS = ["https://docs.databricks.com/aws/en/machine-learning/"]  # example URL list
TXT_PATH = "/content/notes.rtf"
                     # plain text file

# 3) Load documents
def load_docs():
    if SOURCE_TYPE == "pdf":
        loader = PyPDFLoader(PDF_PATH)
        return loader.load()
    elif SOURCE_TYPE == "url":
        loader = WebBaseLoader(URLS)
        return loader.load()
    elif SOURCE_TYPE == "txt":
        loader = TextLoader(TXT_PATH, encoding="utf-8")
        return loader.load()
    else:
        raise ValueError("Unsupported SOURCE_TYPE. Use 'pdf', 'url', or 'txt'.")

docs = load_docs()
print(f"Loaded {len(docs)} document chunks (pre-split).")

# 4) Split into chunks (tune if needed)
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
splits = splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

# 5) Build vector store with cosine similarity
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(
    splits, embedding=emb, distance_strategy=DistanceStrategy.COSINE
)
retriever = vs.as_retriever(search_kwargs={"k": 3})
print("Vector store ready.")

# 6) Prompt with “use context only” + citations
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "If the answer is not in the context, say you don't know.\n"
    "At the end, list citations as [source#page_or_id]."
)

def format_docs(docs):
    lines = []
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        lines.append(f"[{src}#{page}] {d.page_content}")
    return "\n\n".join(lines)

# 7) LLM + chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8) (Optional) Save for reuse later
SAVE_DIR = "/content/my_faiss_index_docs"
vs.save_local(SAVE_DIR)
print("Saved FAISS index to:", SAVE_DIR)

# 9) Q&A loop
print("\n💬 Ask about your document(s). Type 'exit' to quit.")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    ans = rag_chain.invoke(q)
    print("A:", ans, "\n")


Loaded 1 document chunks (pre-split).
Split into 59 chunks.
Vector store ready.
Saved FAISS index to: /content/my_faiss_index_docs

💬 Ask about your document(s). Type 'exit' to quit.
Q: what is gen ai
A: Generative AI (GenAI) is defined as using models (often deep neural networks) to create new content—such as text, images, code, etc.—that closely mimics human-generated data. 

[source#na] 

Q: what is ai
A: AI, or Artificial Intelligence, is defined as the overarching concept of machines that can perform tasks that typically require human intelligence, such as reasoning, decision-making, and perception. [source#na] 

Q: who is SRT
A: I don't know. 

Citations: [14], [15]. 

Q: exit
👋 Goodbye!


In [51]:
# End of the Notebook

In [54]:
# Build RAG Pipeline application via Gradio - UI.

In [52]:
# Example 9 - Gradio Implementation

In [53]:
# ============================================================
# Gradio UI for a Tiny RAG over PDF / URL / TXT / Raw text
# Colab-ready
# ============================================================

!pip install -q gradio langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf bs4

import os, json, tempfile, textwrap
import gradio as gr

from google.colab import userdata
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, WebBaseLoader
from langchain.schema import Document

# ────────────────────────────────────────────────────────────
# API key (expects you've set it with userdata.set(...) once)
# ────────────────────────────────────────────────────────────
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# ────────────────────────────────────────────────────────────
# Helpers
# ────────────────────────────────────────────────────────────
def _format_docs(docs):
    """Format retrieved docs with simple citations."""
    lines = []
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        lines.append(f"[{src}#{page}] {d.page_content}")
    return "\n\n".join(lines)

def _citations(docs):
    """Return a compact citations string like [source#page], deduped."""
    tags = []
    seen = set()
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        tag = f"[{src}#{page}]"
        if tag not in seen:
            tags.append(tag)
            seen.add(tag)
    return " ".join(tags) if tags else "(no citations)"

# ────────────────────────────────────────────────────────────
# Build index: load → chunk → embed → FAISS (cosine)
# ────────────────────────────────────────────────────────────
def build_index(
    source_type,
    files,        # list of temp files from gradio for "Files"
    urls_text,    # str for "URL(s)" separated by newline/comma
    raw_text,     # str for "Raw Text"
    chunk_size,
    chunk_overlap,
    k,
    model_name,
    temperature
):
    try:
        # 1) Load docs
        docs = []
        if source_type == "Files (PDF/TXT)":
            if not files:
                return gr.update(value="Please upload at least one file."), None, None, None
            for f in files:
                # Decide by extension
                name = f.name if hasattr(f, "name") else str(f)
                if name.lower().endswith(".pdf"):
                    loader = PyPDFLoader(name)
                    docs.extend(loader.load())
                else:
                    loader = TextLoader(name, encoding="utf-8")
                    docs.extend(loader.load())

        elif source_type == "URL(s)":
            if not urls_text.strip():
                return gr.update(value="Please enter at least one URL."), None, None, None
            # split on newline or comma
            raw_urls = [u.strip() for u in urls_text.replace(",", "\n").split("\n") if u.strip()]
            loader = WebBaseLoader(raw_urls)
            docs = loader.load()

        elif source_type == "Raw Text":
            if not raw_text.strip():
                return gr.update(value="Please paste some text."), None, None, None
            docs = [Document(page_content=raw_text, metadata={"source": "raw.txt"})]

        else:
            return gr.update(value="Unsupported source type."), None, None, None

        if not docs:
            return gr.update(value="No text found after loading."), None, None, None

        # 2) Chunk
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(chunk_size), chunk_overlap=int(chunk_overlap)
        )
        splits = splitter.split_documents(docs)

        # 3) Embeddings + FAISS (cosine)
        emb = OpenAIEmbeddings(model="text-embedding-3-small")
        vs = FAISS.from_documents(splits, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
        retriever = vs.as_retriever(search_kwargs={"k": int(k)})

        # 4) LLM (GPT model)
        llm = ChatOpenAI(model=model_name, temperature=float(temperature))

        status = (
            f"✅ Index built.\n"
            f"- Chunks: {len(splits)}\n"
            f"- k: {k}\n"
            f"- LLM: {model_name} (temp={temperature})\n"
            f"- Similarity: COSINE (FAISS)"
        )
        return status, vs, retriever, llm

    except Exception as e:
        return f"❌ Error while building index: {e}", None, None, None

# ────────────────────────────────────────────────────────────
# Answer a question from the retriever context
# ────────────────────────────────────────────────────────────
def ask_question(chat_history, question, retriever, llm):
    if retriever is None or llm is None:
        chat_history = chat_history or []
        chat_history.append(("",
            "Please build the index first (choose a source and click 'Build Index')."
        ))
        return chat_history

    try:
        # Retrieve
        ctx_docs = retriever.get_relevant_documents(question)
        context = _format_docs(ctx_docs)

        # Prompt (simple & explicit grounding)
        messages = [
            {"role": "system",
             "content": "You are a helpful assistant. Use ONLY the provided context to answer. "
                        "If the answer is not present, say you don't know."},
            {"role": "user",
             "content": f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"}
        ]
        resp = llm.invoke(messages).content

        # Append citations at the end
        cites = _citations(ctx_docs)
        final = f"{resp}\n\n**Citations:** {cites}"

        chat_history = chat_history or []
        chat_history.append((question, final))
        return chat_history

    except Exception as e:
        chat_history = chat_history or []
        chat_history.append((question, f"❌ Error: {e}"))
        return chat_history

# ────────────────────────────────────────────────────────────
# Gradio UI
# ────────────────────────────────────────────────────────────
with gr.Blocks(title="Tiny RAG UI (FAISS + OpenAI)") as app:
    gr.Markdown("## 🔎 Tiny RAG — Ask questions over your PDF/URL/TXT/Raw text")

    with gr.Accordion("1) Build Index", open=True):
        source_type = gr.Radio(
            ["Files (PDF/TXT)", "URL(s)", "Raw Text"],
            value="Files (PDF/TXT)",
            label="Source type"
        )
        files = gr.File(label="Upload PDF/TXT files", file_types=[".pdf", ".txt"], file_count="multiple", visible=True)
        urls_text = gr.Textbox(label="Enter URL(s) (comma- or newline‑separated)", visible=False, lines=3)
        raw_text = gr.Textbox(label="Paste Raw Text", visible=False, lines=6, placeholder="Paste your text here...")

        with gr.Row():
            chunk_size = gr.Number(label="Chunk size", value=800)
            chunk_overlap = gr.Number(label="Chunk overlap", value=120)
            k = gr.Slider(1, 10, value=3, step=1, label="Top‑k retrieved")

        with gr.Row():
            model_name = gr.Textbox(label="GPT model", value="gpt-4o-mini")
            temperature = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label="Temperature")

        build_btn = gr.Button("🔨 Build Index")
        status = gr.Markdown("Status will appear here.")

    with gr.Accordion("2) Ask Questions", open=True):
        chat = gr.Chatbot(height=320, label="Chat")
        question = gr.Textbox(label="Your question")
        ask_btn = gr.Button("Ask")

    # App state
    vs_state = gr.State(None)
    retriever_state = gr.State(None)
    llm_state = gr.State(None)

    # Source type toggling
    def toggle_inputs(src):
        return (
            gr.update(visible=(src == "Files (PDF/TXT)")),
            gr.update(visible=(src == "URL(s)")),
            gr.update(visible=(src == "Raw Text")),
        )

    source_type.change(
        toggle_inputs, inputs=[source_type], outputs=[files, urls_text, raw_text]
    )

    # Build index
    build_btn.click(
        build_index,
        inputs=[source_type, files, urls_text, raw_text, chunk_size, chunk_overlap, k, model_name, temperature],
        outputs=[status, vs_state, retriever_state, llm_state]
    )

    # Ask
    ask_btn.click(
        ask_question,
        inputs=[chat, question, retriever_state, llm_state],
        outputs=[chat]
    )

# Launch (share=True gives a public URL from Colab)
app.launch(share=True)


/tmp/ipython-input-53-3473171679.py:192: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=320, label="Chat")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://71e4b9d1b852e5164b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
